In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
#load datasets
am_df = pd.read_csv("data/american_stock.csv")
eu_df = pd.read_csv("data/european_index.csv")

In [13]:
am_df.head()


,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,101594,20150102,AAPL 150102C100000,1,20150102,20150102.0,C,100000,9.20,9.35,...,3571,0,4,NaN,0,NaN,APPLE INC,NaN,A,NaN
1,101594,20150102,AAPL 150102C101000,1,20150102,20150102.0,C,101000,8.20,8.40,...,3571,0,4,NaN,0,NaN,APPLE INC,NaN,A,NaN
2,101594,20150102,AAPL 150102C102000,1,20150102,20150102.0,C,102000,7.25,7.35,...,3571,0,4,NaN,0,NaN,APPLE INC,NaN,A,NaN
3,101594,20150102,AAPL 150102C103000,1,20150102,20150102.0,C,103000,6.20,6.40,...,3571,0,4,NaN,0,NaN,APPLE INC,NaN,A,NaN
4,101594,20150102,AAPL 150102C104000,1,20150102,20150102.0,C,104000,5.25,5.35,...,3571,0,4,NaN,0,NaN,APPLE INC,NaN,A,NaN


In [1]:
eu_df.head()

NameError: name 'eu_df' is not defined